# AGREEMENT ANALYSIS

### Initialise packages

In [5]:
import pandas as pd
import statistics

from statsmodels.stats.inter_rater import fleiss_kappa

### Import clean data

In [3]:
df = pd.read_csv('./0_data/clean/davidson2017_clean.csv')

### Compute inter-annotator agreement metrics

**Percentage agreement**

In [9]:
# all data is annotated by three annotators, so agreement is either 2/3 or 3/3
# this information is already saved in the 'unanimous' column

df['pct_agreement'] = df.unanimous.apply(lambda x: 2/3 if x==False else 1)

print('Average percentage agreement:')
df.pct_agreement.mean()

Average percentage agreement:


0.9051899270691972

**Fleiss' Kappa**

In [11]:
vote_matrix = df[['n_hateful','n_offensive', 'n_neither']].to_numpy()

print('Fleiss\' Kappa:')
fleiss_kappa(vote_matrix)

Fleiss' Kappa:


0.5494695584402823

### Compute bootstrap standard errors for agreement metrics

In [12]:
def bootstrap_se(df, runs):
    
    results_dict={}
    
    # MEANS
    
    # RAW AGREEMENT
    results_dict['mean_raw_agreement']={}
    results_dict['mean_raw_agreement']['abs value'] = df['pct_agreement'].mean()
    
    # FLEISS KAPPA
    results_dict['fleiss_kappa']={}
    results_dict['fleiss_kappa']['abs value'] = fleiss_kappa(df[['n_hateful','n_offensive', 'n_neither']].to_numpy())
    
    
    # BOOTSTRAP
    
    metric_list = []
    for i in range(runs):
        b_df = df.sample(frac=1, replace=True)
        metric_list.append(b_df['pct_agreement'].mean())
    
    results_dict['mean_raw_agreement']['bootstrap_sd'] = statistics.pstdev(metric_list)
    
    metric_list = []
    for i in range(runs):
        b_df = df.sample(frac=1, replace=True)
        metric_list.append(fleiss_kappa(b_df[['n_hateful','n_offensive', 'n_neither']].to_numpy()))
    
    results_dict['fleiss_kappa']['bootstrap_sd'] = statistics.pstdev(metric_list)
    
    return results_dict

In [14]:
n = 2000
runs = 500

bootstrap_se(df.sample(n, random_state=123), runs = runs)

{'mean_raw_agreement': {'abs value': 0.9036666666666736,
  'bootstrap_sd': 0.0033223808564874576},
 'fleiss_kappa': {'abs value': 0.542276288525904,
  'bootstrap_sd': 0.014908332137299578}}